In [1]:
import pandas as pd
import numpy as np
import json
import ast
import gzip

In [10]:
pip install textblob

     ---------------------------------------- 0.0/636.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/636.8 kB ? eta -:--:--
      --------------------------------------- 10.2/636.8 kB ? eta -:--:--
      --------------------------------------- 10.2/636.8 kB ? eta -:--:--
     - ----------------------------------- 30.7/636.8 kB 217.9 kB/s eta 0:00:03
     -- ---------------------------------- 41.0/636.8 kB 219.4 kB/s eta 0:00:03
     ---- -------------------------------- 81.9/636.8 kB 353.1 kB/s eta 0:00:02
     -------------- --------------------- 256.0/636.8 kB 983.0 kB/s eta 0:00:01
     ----------------------------- -------- 491.5/636.8 kB 1.6 MB/s eta 0:00:01
     -------------------------------------  634.9/636.8 kB 1.8 MB/s eta 0:00:01
     -------------------------------------- 636.8/636.8 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ----- ---------------------------------- 0.2/1.5 MB 3.9 MB/s eta 0:00:

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


1.Carga de los archivos 

Carga de df output_steam_games

In [52]:


# Lee el archivo JSON y carga los datos en un DataFrame
df_games = pd.read_json(r'D:\Users\Natalia\Desktop\steam_games.json\output_steam_games.json', lines=True)

df_games.head()


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Carga de australian_user_reviews

Por tener columnas anidadas el meetodo usado en df_games no funciona correctamente, por lo que se utiliza este metodo iterando por el archivo, agrgandolo en un alista y finalmente en un df

In [2]:
data = []

# Abre el archivo JSON en modo lectura
with open("D:\\Users\\Natalia\\Desktop\\steam_games.json\\australian_user_reviews.json", "r", encoding="utf-8") as archivo:
    for linea in archivo:
        # Lee cada línea del archivo como una cadena JSON
        linea_json = ast.literal_eval(linea.strip())
        
        # Agrega el diccionario JSON a la lista
        data.append(linea_json)

# Crea el DataFrame directamente desde la lista de diccionarios
df_reviews = pd.DataFrame(data)

In [23]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [24]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [25]:
#columna anidada
type(df_reviews["reviews"])

pandas.core.series.Series

In [3]:
#Se utiliza el método explode() para desglosar la columna 'review' que contiene elementos anidados, 
df_reviews_explode = df_reviews.explode('reviews')
type(df_reviews_aplanado["reviews"])

NameError: name 'df_reviews' is not defined

In [4]:
# desanidamos la columna reviews y eliminamos los nulos
df_reviews_desanidado = pd.json_normalize(df_reviews_explode['reviews'].dropna())

# reindexamos los df para unirlos 
df_reviews_desanidado.reset_index(inplace=True)
df_reviews_explode.reset_index(inplace=True)

# concatenamos y eliminamos la columna anidada original 'reviews'
df_reviews = pd.concat([df_reviews_explode,df_reviews_desanidado], axis=1)
df_reviews = df_reviews.drop(columns = ['reviews'])


In [5]:
df_reviews.head()

,index,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,1,js41637,http://steamcommunity.com/id/js41637,3.0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,1,js41637,http://steamcommunity.com/id/js41637,4.0,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Carga australian_users_items

Al ser el archivo más pesado, el mismo metodo utilizado para df_reviews no funciona correctamente tardando demasiado tiempo y dando error. 
Tampoco lo hace le utilzado en df_games por tener columnas anidadas. Descomprimir el archivo directamente aqui funciona y evita problemas de carga 
con las columnas.

In [3]:
jso_item = r'D:\Users\Natalia\Desktop\steam_games.json\users_items.json.gz'

lista=[]

with gzip.open(jso_item, 'rt', encoding='utf8') as file:
    for line in file.readlines():
        lista.append(ast.literal_eval(line))

In [4]:
df_items = pd.DataFrame(lista)

In [5]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [6]:
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [5]:
#aplano columna items
df_items_aplanado = df_items.explode('items')

In [6]:
# desanidamos la columna 'items'
df_items_desanidado = pd.json_normalize(df_items_aplanado['items'].dropna())

# reindexamos para poder unir los df
df_items_desanidado.reset_index(inplace=True)
df_items_aplanado.reset_index(inplace=True)

# concatenamos y eliminamos la columna anidada original
df_items = pd.concat([df_items_aplanado,df_items_desanidado], axis=1)



In [9]:
#eliminamos la columna 'items' una vez ya desanidada
df_items = df_items.drop(columns = ['items'])

In [10]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 10 columns):
 #   Column            Dtype  
---  ------            -----  
 0   index             int64  
 1   user_id           object 
 2   items_count       int64  
 3   steam_id          object 
 4   user_url          object 
 5   index             float64
 6   item_id           object 
 7   item_name         object 
 8   playtime_forever  float64
 9   playtime_2weeks   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 394.4+ MB


In [11]:
df_items.head()

,index,user_id,items_count,steam_id,user_url,index,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,10,Counter-Strike,6.0,0.0
1,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,20,Team Fortress Classic,0.0,0.0
2,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,30,Day of Defeat,7.0,0.0
3,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3.0,40,Deathmatch Classic,0.0,0.0
4,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4.0,50,Half-Life: Opposing Force,0.0,0.0


2.Limpieza de df y los redusco a las culumnas necesarias para las funciones

Elimino las columnas inecesarias.

Decido eliminar ´tags' quedandome solo con generos ya que muchos de estos valores se repiten, y la clasificacion por tags al ser definia por usuarios es menos constante y más subjetiva, mientras que la clasificacion por 'genero' la podemos considerar como oficial y es más constante

In [53]:
df_games_limpio = df_games.drop(["publisher","url","tags","reviews_url","specs","price","early_access","developer"],axis=1)

In [7]:
df_games_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   app_name      32133 non-null  object 
 2   title         30085 non-null  object 
 3   release_date  30068 non-null  object 
 4   id            32133 non-null  float64
dtypes: float64(1), object(4)
memory usage: 4.6+ MB


In [45]:
df_games_limpio.head()

,genres,app_name,title,release_date,id
0,None,None,None,None,NaN
1,None,None,None,None,NaN
2,None,None,None,None,NaN
3,None,None,None,None,NaN
4,None,None,None,None,NaN


In [49]:
df_games_limpio.isnull().sum()

genres          91593
app_name        88312
title           90360
release_date    90377
id              88312
dtype: int64

Considerando al gran cantidad de archivos nulos en el df los elimino. Tanto las columnas 'title' como 'app_name' tienen informacion solbre el titulo del juego, pero considerando que 'title' tiene más elementos nulos, 'app_name' parece ser la mas completa. Por eso elimino 'title' del df y eliminos los 
valores nulos tomando 'app_name' como base

In [54]:
df_games_limpio.drop(["title"],axis=1, inplace=True)

In [55]:
df_games_limpio.dropna(subset=['app_name'], inplace=True)


In [56]:
df_games_limpio.head()

,genres,app_name,release_date,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140.0
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980.0
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290.0
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400.0
88314,None,Log Challenge,None,773570.0


Analiso la columna 'genres' y la desgloso para tener lso generos como columnas individuales con valores binarios 0 y 1

In [12]:
df_games_limpio['genres'].explode().unique()



array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', None, 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [64]:
df_games_limpio['genres'] = df_games_limpio['genres'].fillna('[]')  
df_games_limpio['genres'] = df_games_limpio['genres'].apply(lambda x: ', '.join(x)) 

_genres = df_games_limpio['genres'].str.get_dummies(', ')

df_games_limpio = pd.concat([df_games_limpio, _genres], axis=1)

In [ ]:
df_games_limpio.drop(columns=["[","]","genres"],inplace=True)

In [ ]:
df_games_limpio.head()

limpio la columna de 'release date' para tener el año correctamente

In [ ]:
df_games_limpio['release_date'].unique()

In [ ]:
# Verificar cuántas fechas cumplen con el formato esperado (sin incluir nulos)
formato_esperado = "^\d{4}-\d{2}-\d{2}$"
fechas_con_formato = df_games_limpio["release_date"].str.match(formato_esperado).sum()

# Obtener la cantidad total de fechas no nulas
total_fechas = df_games_limpio["release_date"].count()

# Calcular la cantidad de fechas que no siguen el formato, excluyendo los valores nulos
cantidad_fechas_no_formato = total_fechas - fechas_con_formato

print(f"Cantidad de fechas que no siguen el formato esperado : {cantidad_fechas_no_formato}")

al compara la camtidad de fechas qeu no coinciden en fromato (285) vemos que no hay mucha diferencia conel nuemro de elemntos nuelos en esa columna (2066).
por lo que conviento estos valroes qeu no cinciden a nulos o vacios. Decido no eliminar esto svalores para no modificar el respecto de las columnas

In [58]:
#llenamos los espacios nulos por lugares vacios
df_games_limpio["release_date"].fillna("", inplace=True)

# Identifico las fechas que no coinciden con el formato YYYY-MM-DD
invalid_dates = df_games_limpio[~df_games_limpio["release_date"].str.match("^\d{4}-\d{2}-\d{2}$")]

# Reemplaza las fechas inválidas por valores vacíos en la columna "release_date"
df_games_limpio.loc[invalid_dates.index, "release_date"] = ""

# Convierte la columna "release_date" al formato datetime
df_games_limpio["release_date"] = pd.to_datetime(df_games_limpio["release_date"], errors="coerce")


In [ ]:
df_games_limpio.loc[:,['release_date']]

ahora creamos una nueva columna donde solo esta el año utilizando strftime("%Y") para no tener valores flotantes en los años
como ocurre al utilizar dt.year

In [60]:
df_games_limpio["año"]=df_games_limpio["release_date"].dt.strftime("%Y")
#dt.strftime("%Y") para garantizar la representación de cadenas de texto para los años en lugar de valores numéricos.

In [61]:
#elimino columna release date
df_games_limpio = df_games_limpio.drop(['release_date'],axis=1)

In [62]:

# cambiear el tipo de dato de id para que no sea float
# Reemplazar valores no finitos (NaN o infinito) con NaN
df_games_limpio['id'] = df_games_limpio['id'].replace([np.inf, -np.inf], np.nan)

# Convertir la columna 'id' a tipo entero
df_games_limpio['id'] = df_games_limpio['id'].astype(float).astype(pd.Int32Dtype())




Finalmente con el dataset limpio y las columnas ya preparadas, lo paso a csv para que se más sensillo trabajar con las funciones posteriormente

In [70]:
df_games_limpio.to_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_games.csv', index=False, float_format='%.0f', na_rep='NaN')


limpiesa de df_items

In [16]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 10 columns):
 #   Column            Dtype  
---  ------            -----  
 0   index             int64  
 1   user_id           object 
 2   items_count       int64  
 3   steam_id          object 
 4   user_url          object 
 5   index             float64
 6   item_id           object 
 7   item_name         object 
 8   playtime_forever  float64
 9   playtime_2weeks   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 394.4+ MB


In [17]:
# elimino columnas inecesarias
df_items_limpio = df_items.drop(["index","items_count","steam_id","user_url","playtime_2weeks"],axis=1)


In [18]:
df_items_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           object 
 2   item_name         object 
 3   playtime_forever  float64
dtypes: float64(1), object(3)
memory usage: 157.8+ MB


In [19]:
df_items_limpio.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0


In [20]:
df_items_limpio.isnull().sum()

user_id                 0
item_id             16806
item_name           16806
playtime_forever    16806
dtype: int64

Ya que la cantidad de nulos es unifrome en todo el df considero que al eliminarlos no pierdo demasiada informacion, por lo que elimino todos

In [21]:
df_items_limpio.dropna(inplace=True)


una ves limpio guado el dataset en csv para facilitar su manejo

In [22]:
df_items_limpio.to_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_items.csv', index=False)


Limpiesa de df_reviews

In [77]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        59333 non-null  int64  
 1   user_id      59333 non-null  object 
 2   user_url     59333 non-null  object 
 3   index        59305 non-null  float64
 4   funny        59305 non-null  object 
 5   posted       59305 non-null  object 
 6   last_edited  59305 non-null  object 
 7   item_id      59305 non-null  object 
 8   helpful      59305 non-null  object 
 9   recommend    59305 non-null  object 
 10  review       59305 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 5.0+ MB


In [79]:
df_reviews.head()

,index,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,1,js41637,http://steamcommunity.com/id/js41637,3.0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,1,js41637,http://steamcommunity.com/id/js41637,4.0,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [6]:
#elimino columnas inecesarias
df_reviews_columnas = df_reviews.drop(["index","index","last_edited","funny","helpful","user_url"], axis=1)


In [7]:
df_reviews_columnas.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [83]:
#veo cantidad de nullos
df_reviews_columnas.isnull().sum()

user_id       0
posted       28
item_id      28
recommend    28
review       28
dtype: int64

Elimino los nullos, como son pocos y la mima cantidad en todo los casos lo hago directamete

In [8]:

df_reviews_columnas.dropna(inplace=True)

Agrego la columna 'sentiment_analysis'

In [11]:
from textblob import TextBlob
#importo el modulo para tomar el texto en la columna review, analizarlo y pasarlo a valores 

In [35]:
df_reviews_limpio= df_reviews_columnas.copy() 
#hago una copia del df por seguridad 

In [36]:
df_reviews_limpio.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [37]:
#funcion para crear la columna 'sentiment analysis'
def obtener_valor_sentimiento(frase):
    analysis = TextBlob(frase)
    if analysis.sentiment.polarity < 0:
        return 0  # Malo
        """""
        si la polaridad de la frse se identifica como menor a 0 devuelve el valor 0, identificado como malo
        """
    elif analysis.sentiment.polarity == 0:
        return 1  # Neutral
        """""
        si la polaridad de la frse se identifica como = a 0 devuelve el valor 0, identificado como neutral
        """
    else:
        return 2  # Positivo
        """""
        si no curre ninguna de las anteriores,
         la polaridad de la frse se identifica como mayor a 0 devuelve el valor 2, identificado como positivo
        """


In [38]:
#se aplica la funcion directamente sobre la columna 'review' en la creacion de la nueva columna 'sentiment_analysis'
df_reviews_limpio['sentiment_analysis'] = df_reviews_limpio['review'].apply(obtener_valor_sentimiento)

In [45]:
#se elimna la columna 'review'
df_reviews_limpio = df_reviews_limpio.drop('review', axis=1)

In [46]:
df_reviews_limpio.head()

,user_id,item_id,recommend,sentiment_analysis,año_publicado
0,76561197970982479,1250,True,2,2011
1,76561197970982479,22200,True,2,2011
2,76561197970982479,43110,True,2,2011
3,js41637,251610,True,2,2014
4,js41637,227300,True,0,2013


Modifico la columna 'posted' para extraer solo el año. Los valores donde no figura el año se ignoran quedando como nulos. No los elimino 
para no modificar el resto de las columnas

In [22]:
df_reviews_limpio['posted']

0         Posted November 5, 2011.
1            Posted July 15, 2011.
2           Posted April 21, 2011.
3            Posted June 24, 2014.
4        Posted September 8, 2013.
                   ...            
59300              Posted July 10.
59301               Posted July 8.
59302               Posted July 3.
59303              Posted July 20.
59304               Posted July 2.
Name: posted, Length: 59305, dtype: object

In [40]:
 #se extraen los valores que sigan el patron de año 
df_reviews_limpio["año_publicado"]=df_reviews["posted"].str.extract(r'(\d{4})')

In [42]:
df_reviews_limpio = df_reviews_limpio.drop("posted", axis=1)

In [51]:
df_reviews_limpio.head()

,user_id,item_id,recommend,sentiment_analysis,año_publicado
0,76561197970982479,1250,True,2,2011
1,76561197970982479,22200,True,2,2011
2,76561197970982479,43110,True,2,2011
3,js41637,251610,True,2,2014
4,js41637,227300,True,0,2013


In [47]:
df_reviews_limpio.isnull().sum()

user_id                   0
item_id                   0
recommend                 0
sentiment_analysis        0
año_publicado         10119
dtype: int64

In [32]:
df_reviews_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59304
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   posted              59305 non-null  object
 2   item_id             59305 non-null  object
 3   recommend           59305 non-null  object
 4   review              59305 non-null  object
 5   sentiment_analysis  59305 non-null  int64 
 6   año_publicado       49186 non-null  object
dtypes: int64(1), object(6)
memory usage: 3.6+ MB


Finalmente con el df_review_limpio en condiciones contodas las columnas necesarias lo paso a un csv para facilitar su carga al trabajar con las funciones

In [49]:
df_reviews_limpio.to_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_reviews.csv', index=False,float_format='%.0f')
#las indicaciones finales son para evitar cambios en el fromato de los valores al pasarse a csv

con los datasets solo con las columnas necesarias paso a preparar los datos para porteriromente pder accedera a esot mediante las funciones
finalmente paso esto snuevos datasets a csv para facilitar su carga y manejo

primero modificare el datasets games, dividiendo la columna 'generos' creando distintas colmnas con cada uno de los generos, 
utilizando variables binarias

In [6]:
df_games_limpio.isnull().sum()

genres          3282
app_name           0
release_date    2066
id                 1
dtype: int64

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2016-11-19',
       'January 2018', '2018-10-01'], dtype=object)

In [53]:
df_games_limpio['release_date'].count()

32133

In [59]:
df_games_limpio.head()

,genres,app_name,release_date,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140.0
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980.0
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290.0
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400.0
88314,None,Log Challenge,NaT,773570.0


In [63]:
df_games_limpio.head()

,genres,app_name,id,año
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,761140,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,643980,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,670290,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,767400,2017
88314,None,Log Challenge,773570,NaN


ahora separar al columna de generos para tenerlos como variables individuales

In [33]:
df_games_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    28851 non-null  object
 1   app_name  32133 non-null  object
 2   id        32132 non-null  Int32 
 3   año       29782 non-null  object
dtypes: Int32(1), object(3)
memory usage: 2.1+ MB


In [32]:
df_games_limpio['genres'].explode().unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', None, 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [13]:
#backup del dataset original
df_games_limpio_b = df_games_limpio.copy()

In [69]:
df_games_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   app_name                   32133 non-null  object
 1   id                         32132 non-null  Int32 
 2   año                        29782 non-null  object
 3   Accounting                 32133 non-null  int64 
 4   Action                     32133 non-null  int64 
 5   Adventure                  32133 non-null  int64 
 6   Animation &amp; Modeling   32133 non-null  int64 
 7   Audio Production           32133 non-null  int64 
 8   Casual                     32133 non-null  int64 
 9   Design &amp; Illustration  32133 non-null  int64 
 10  Early Access               32133 non-null  int64 
 11  Education                  32133 non-null  int64 
 12  Free to Play               32133 non-null  int64 
 13  Indie                      32133 non-null  int64 
 14  Massiv